In [ ]:
from prj.config import DATA_DIR
from prj.data.data_loader import DataConfig, DataLoader
import polars as pl
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from prj.data.data_loader import PARTITIONS_DATE_INFO
import pandas as pd
import lightgbm as lgb
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import gc
from tqdm import tqdm
import numpy as np
from prj.utils import online_iterator, online_iterator_daily
import time
from sklearn.model_selection import train_test_split
SEED = 42
import lleaves

2025-01-06 18:36:58.503438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-06 18:36:58.503497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-06 18:36:58.504681: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-06 18:36:58.511599: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-06 18:36:59.856753: W tensorflow/compiler/tf2

: 

# Base

In [ ]:
use_weighted_loss = False
params = {
    'verbose': 50,
    'iterations': 717,
    'learning_rate': 0.019678599283449602,
    'depth': 8,
    'has_time': False,
    'bootstrap_type': 'Bernoulli',
    'reg_lambda': 0.00924440304487912,
    'min_data_in_leaf': 72,
    'subsample': 0.63603957073985,
    'task_type': 'GPU',
}

model = CatBoostRegressor(**params)

config = DataConfig(**{'include_intrastock_norm_temporal': True, 'include_time_id': True})
loader = DataLoader(data_dir=DATA_DIR, config=config)

start_dt, end_dt = 680, 1359
train_df = loader.load(start_dt, end_dt)
X_train, y_train, w_train, _ = loader._build_splits(train_df)

model.fit(
    X_train,
    y_train,
    sample_weight=w_train if use_weighted_loss else None,
)

100%|██████████| 687/687 [00:41<00:00, 16.52it/s]


Skipping 1353-1359
Skipping 1354-1359
Skipping 1355-1359
Skipping 1356-1359
Skipping 1357-1359
Skipping 1358-1359
Skipping 1359-1359


In [ ]:
model.save_model('catboost_model_4_7.cbm')

In [16]:
del X_train, y_train, w_train
gc.collect()

0

In [17]:
start_dt, end_dt = 1360, 1529
train_df = loader.load(start_dt, end_dt)
X_test, y_test, w_test, _ = loader._build_splits(train_df)

100%|██████████| 177/177 [00:14<00:00, 12.54it/s]


Skipping 1523-1529
Skipping 1524-1529
Skipping 1525-1529
Skipping 1526-1529
Skipping 1527-1529
Skipping 1528-1529
Skipping 1529-1529


In [18]:
y_hat  = model.predict(X_test)
r2_score(y_test, y_hat, sample_weight=w_test)

0.014105837093202789

In [19]:
start_dt, end_dt = 1530, 1698
train_df = loader.load(start_dt, end_dt)
X_test, y_test, w_test, _ = loader._build_splits(train_df)

y_hat  = model.predict(X_test)
r2_score(y_test, y_hat, sample_weight=w_test)

100%|██████████| 176/176 [00:14<00:00, 12.13it/s]


Skipping 1692-1698
Skipping 1693-1698
Skipping 1694-1698
Skipping 1695-1698
Skipping 1696-1698
Skipping 1697-1698
Skipping 1698-1698


0.007757179180381324

In [20]:
# model.save_model('catboost_model.cbm')

In [ ]:
# 0.013016677344887873